# **Global settings**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import scrublet as scr
import scvelo as scv
import seaborn as sns
import scipy.io
import os
import dotplot
import dotplot.utils
import math
import gseapy as gp
from gseapy.plot import barplot, dotplot, gseaplot
from gseapy.scipalette import SciPalette
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap 

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, figsize=(5, 5))

In [ ]:
plt.set_cmap('viridis')

<Figure size 500x500 with 0 Axes>

In [ ]:
scv.set_figure_params()

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc')

## **RNA velocity analysis**

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc/velocyto')

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
ap={}
al={}
sample=['Sample_1_SI_0','Sample_2_SI_0','Sample_3_SI_0','Sample_4_SI_60','Sample_5_SI_60',
        'Sample_6_SI_60','Sample_7_SI_90','Sample_8_SI_90','Sample_9_SI_90','Sample_10_SI_180',
        'Sample_11_SI_180','Sample_12_SI_180','Sample_13_SI_240','Sample_14_SI_240','Sample_15_SI_240',
        'Sample_16_LI_0','Sample_17_LI_0','Sample_18_LI_60','Sample_19_LI_60','Sample_20_LI_90',
        'Sample_21_LI_90','Sample_22_LI_180','Sample_23_LI_180','Sample_24_LI_240','Sample_25_LI_240']

for i in sample:
    ap[f'{i}']=sc.read_loom(PATH+f'{i}'+'/'+f'{i}'+'.loom',sparse=True)
    ap[f'{i}'].var_names_make_unique
    length=len(f'{i}')+1
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.str[length:-1]
    x=['{}','1']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('-'.join(x).format)
    y=['{}',f'{i}']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('_'.join(y).format)
    if(ap[f'{i}'].obs.shape[1]!=0):
        ap[f'{i}'].obs=ap[f'{i}'].obs.drop(columns=['Clusters','_X', '_Y'])
    al[f'{i}']=scv.utils.merge(Epithelial[Epithelial.obs['PRO1_JH'].isin([f'{i}'])], ap[f'{i}'])

In [ ]:
ldata=sc.concat(ap.values(),keys=ap.keys(),label='loom_JH')
ldata

In [ ]:
Epithelial=scv.utils.merge(Epithelial, ldata)
Epithelial

In [ ]:
Epithelial.write('Epithelial_velocyto_merge.h5ad')

In [ ]:
#scv.pp.filter_and_normalize(Epithelial, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(Epithelial)

In [ ]:
scv.tl.velocity(Epithelial)

In [ ]:
scv.tl.velocity_graph(Epithelial)

In [ ]:
scv.pl.velocity_embedding_stream(Epithelial, basis='umap', color='CellType', legend_loc='right margin')
scv.pl.velocity_embedding_stream(Epithelial, basis='umap', color='CellType', legend_loc='right margin', alpha=0, show=False, save='stream_Epithelial_right_margin.svg') 

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
sample=['Sample_1_SI_0','Sample_2_SI_0','Sample_3_SI_0','Sample_4_SI_60','Sample_5_SI_60',
        'Sample_6_SI_60','Sample_7_SI_90','Sample_8_SI_90','Sample_9_SI_90','Sample_10_SI_180',
        'Sample_11_SI_180','Sample_12_SI_180','Sample_13_SI_240','Sample_14_SI_240','Sample_15_SI_240',
        'Sample_16_LI_0','Sample_17_LI_0','Sample_18_LI_60','Sample_19_LI_60','Sample_20_LI_90',
        'Sample_21_LI_90','Sample_22_LI_180','Sample_23_LI_180','Sample_24_LI_240','Sample_25_LI_240']

for i in sample:
    al[f'{i}'].write(PATH+f'{i}'+'/'+'Epithelial_velocyto_'+f'{i}'+'.h5ad')

In [ ]:
for i in sample:
    newdata=al[f'{i}']
    scv.pp.moments(newdata)
    scv.tl.velocity(newdata)
    scv.tl.velocity_graph(newdata)
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='CellType', show=False, save='grid_Epithelial_'+f'{i}'+'.pdf')
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='CellType', alpha=0, show=False, save='grid_Epithelial_alpha0_'+f'{i}'+'.pdf') #网格线